<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/Kiswahili_Bytepair_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kiswahili BytePair Tokenizer**

**Introduction**

*   Kiswahili has a different strcture to English or any other well-researched language in NLP. 
*   It is therefore, important to design a tokenizer from scratch for Kiswahili to meet its NLP needs.



# **0.0 Libraries**

In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import pandas as pd
import numpy as np
import datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.8 MB/s eta 0:00:00


In [3]:
import transformers

# **1.0 Loading the Datasets**

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/train.csv')
print(df)

            id                                            content category
0       SW4670   Bodi ya Utalii Tanzania (TTB) imesema, itafan...   uchumi
1      SW30826   PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ...  kitaifa
2      SW29725  Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...   uchumi
3      SW20901   TIMU ya taifa ya Tanzania, Serengeti Boys jan...  michezo
4      SW12560   Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...  kitaifa
...        ...                                                ...      ...
23263  SW24920   Alitoa pongezi hizo alipozindua rasmi hatua y...   uchumi
23264   SW4038   Na NORA DAMIAN-DAR ES SALAAM  TEKLA (si jina ...  kitaifa
23265  SW16649   Mkuu wa Mkoa wa Njombe, Dk Rehema Nchimbi wak...   uchumi
23266  SW23291   MABINGWA wa Ligi Kuu Soka Tanzania Bara, Simb...  michezo
23267  SW11778   WIKI iliyopita, nilianza makala haya yanayole...  kitaifa

[23268 rows x 3 columns]


# **2.0 Building The Tokenizer**

**The Process Used**

Tokenization involves 5 steps:


*   Normalization - text cleanup such as lowercasing, removing accents etc.
*   Pre-tokenization - splitting the words.
*   Model - the actual tokenization.
*   Post-processing - Special tokens are added.
*   Decoder - the final step, tokenized data converted to human-readable text.



## **2.1 Libraries**

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers import models

In [6]:
tokenizer = Tokenizer(models.BPE())

No need to specify an unk_token because GPT-2 uses byte-level BPE, which doesn’t require it.

## **2.2 Normalization**

GPT-2 does not use a normalizer, so we skip that step and go directly to the pre-tokenization.

## **2.3 Pre-Tokenization**

In [8]:
from tokenizers import pre_tokenizers
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

## **2.4 Training the Tokenizer**

In [9]:
from tokenizers import trainers

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])


In [10]:
tokenizer.train_from_iterator(df, trainer=trainer)

## **2.5 Post Processing**

In [11]:
from tokenizers import processors

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

## **2.6 Decoder**

In [12]:
from tokenizers import decoders

tokenizer.decoder = decoders.ByteLevel()

## **2.7 Saving the Tokenizer**

In [13]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)
# save the tokenizer
wrapped_tokenizer.save_pretrained('/content/Kisw_bytepair_tokn')

('/content/Kisw_bytepair_tokn/tokenizer_config.json',
 '/content/Kisw_bytepair_tokn/special_tokens_map.json',
 '/content/Kisw_bytepair_tokn/tokenizer.json')

# **3.0 Using the Tokenizer**

In [14]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('/content/Kisw_bytepair_tokn')

In [15]:
tokens = tokenizer("Ilikuwa wakati wa jioni jua limepunguza udhia wake na upepo mwanana ulikuwa ukipita na kuzipapasa ngozi zetu mfano wa pamba")

In [16]:
tokens

{'input_ids': [6, 1, 1, 13, 6, 1, 6, 8, 7, 6, 1, 6, 4, 7, 5, 1, 3, 6, 1, 1, 4, 7, 1, 4, 8, 1, 7, 1, 7, 1, 6, 1, 6, 6, 10, 1, 7, 1, 6, 1, 1, 1, 7, 5, 8, 6, 4, 10, 1, 7, 8, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='/content/Kisw_bytepair_tokn', vocab_size=25, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>'})


In [18]:
tokens.input_ids 

[6,
 1,
 1,
 13,
 6,
 1,
 6,
 8,
 7,
 6,
 1,
 6,
 4,
 7,
 5,
 1,
 3,
 6,
 1,
 1,
 4,
 7,
 1,
 4,
 8,
 1,
 7,
 1,
 7,
 1,
 6,
 1,
 6,
 6,
 10,
 1,
 7,
 1,
 6,
 1,
 1,
 1,
 7,
 5,
 8,
 6,
 4,
 10,
 1,
 7,
 8,
 1,
 1,
 1]

Ref: https://huggingface.co/course/chapter6/8?fw=pt